In [1]:
from typing import *

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python import keras

import os

from lib.dnn.utils import KerasTrainer, Optimizer
from lib.dnn.layers import Delta, FloatEmbedding


2021-09-09 05:03:12.748506: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-09 05:03:12.748551: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class Trial0Trainer(KerasTrainer):

	def __init__(self, seq_len, hidden_layers: List[int], *args, loss="binary_crossentropy", optimizer="adam", hidden_activation="relu", **kwargs):
		super(Trial0Trainer, self).__init__(*args, **kwargs)
		self.__seq_len = seq_len
		self.__hidden_layers = hidden_layers
		self.__loss = loss
		self.__optimizer = optimizer
		self.__hidden_activation = hidden_activation

	def __get_currency_pairs(self, df: pd.DataFrame) -> List[Tuple[str, str]]:
		DELIMITER = "/"
		return [(pair.split(DELIMITER)[0], pair.split(DELIMITER)[1]) for pair in set(df["base_currency"] + DELIMITER + df["quote_currency"])]# TODO FIND A CLEAR WAY

	def __prepare_for_pair(self, sequence: np.ndarray, seq_len: int):
		data_len = sequence.shape[0] - seq_len
		X = np.zeros((data_len, seq_len))
		y = np.zeros((data_len,))
		for i in range(data_len):
			X[i] = sequence[i:i+seq_len]
			if sequence[i] > sequence[i-1]:
				y[i] = 1
			else:
				y[i] = 0
		return X, y
		
	def _prepare_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
		currency_pairs = self.__get_currency_pairs(data)
		
		X = []
		y = []
	
		for base_currency, quote_currency in currency_pairs:
			pair_sequence = data[data["base_currency"] == base_currency][data[data["base_currency"] == base_currency]["quote_currency"] == quote_currency]["c"].to_numpy() # TODO
			pair_X, pair_y = self.__prepare_for_pair(pair_sequence, self.__seq_len)
			X += list(pair_X)
			y += list(pair_y)
		
		
		return np.array(X), np.array(y)
			
	def _create_model(self) -> keras.Model:
		input_layer = keras.layers.Input(shape=self.__seq_len)
		
		output_layer = keras.layers.Dense(1, activation="softmax")
		
		if len(self.__hidden_layers) == 0:
			output_layer = output_layer(input_layer)
		else:
			hidden_layer = keras.layers.Dense(self.__hidden_layers[0], activation=self.__hidden_activation)(input_layer)
			for layer_size in self.__hidden_layers[1:]:
				hidden_layer = keras.layers.Dense(layer_size, activation=self.__hidden_activation)(hidden_layer)
			output_layer = output_layer(hidden_layer)
		
		model = keras.Model(inputs=input_layer, outputs=output_layer)
		model.compile(optimizer=self.__optimizer, loss=self.__loss, metrics=["accuracy"])
		
		return model


In [3]:
class Trial0Optimizer(Optimizer):
	
	def __init__(self, max_power, max_depth, base=4, min_power=3):
		self.__max_power = max_power
		self.__max_depth = max_depth
		self.__base = base
		self.__min_power = min_power
		super(Trial0Optimizer, self).__init__()
	
	def __generate_possible_layers(self, layer_sizes, depth):
		if depth == 1:
			return [[size] for size in layer_sizes]
		
		layers = []
		returned_layers = self.__generate_possible_layers(layer_sizes, depth-1)
		layers += returned_layers
		for size in layer_sizes:
			for layer in returned_layers:
				layers += [[size] + layer]
		
		return layers
		
	def __generate_hidden_layers(self, base, min_power, max_power, max_depth) -> List[List[int]]:
		print("[+]Generating Hidden Layer Sizes...")
		
		layer_sizes = [base ** i for i in range(min_power, max_power)]
		
		return self.__generate_possible_layers(layer_sizes, max_depth)
		
	def _generate_param_values(self) -> Dict:
		return {
			"seq_len": [2**(i+2) for i in range(6)],
			"hidden_layers": self.__generate_hidden_layers(self.__base, self.__min_power, self.__max_power, self.__max_depth),
			"loss": ["binary_crossentropy"],
			"optimizer": ["adam", "sgd"],
			"hidden_activation": ["relu", "tanh"]
		}
		
	def _create_trainer(self, params) -> KerasTrainer:
		print("[+]Creating Trainer...")
		return Trial0Trainer(**params, export_path="temp/dummyModel.h5", data_path="temp/Data/All-All.csv", epochs=2)


In [4]:
MAX_POWER = 6
MIN_POWER = 3
BASE = 4


In [5]:
trainer = Trial0Trainer(10, [2, 3], data_path="temp/Data/All-All.csv", export_path="Model.h5")

In [6]:
#trainer.start()

In [7]:
optimizer = Trial0Optimizer(2, 1, 4, 1)


[+]Generating Hidden Layer Sizes...


In [8]:
optimizer.optimize()

[+]Starting Optimization...
[+]Optimizing Params: ['seq_len', 'hidden_layers', 'loss', 'optimizer', 'hidden_activation'] with Default Values: {}
[+]Optimizing Params: ['hidden_layers', 'loss', 'optimizer', 'hidden_activation'] with Default Values: {'seq_len': 4}
[+]Optimizing Params: ['loss', 'optimizer', 'hidden_activation'] with Default Values: {'seq_len': 4, 'hidden_layers': [4]}
[+]Optimizing Params: ['optimizer', 'hidden_activation'] with Default Values: {'seq_len': 4, 'hidden_layers': [4], 'loss': 'binary_crossentropy'}
[+]Optimizing Params: ['hidden_activation'] with Default Values: {'seq_len': 4, 'hidden_layers': [4], 'loss': 'binary_crossentropy', 'optimizer': 'adam'}
[+]Getting Loss for :{'seq_len': 4, 'hidden_layers': [4], 'loss': 'binary_crossentropy', 'optimizer': 'adam', 'hidden_activation': 'relu'}
[+]Creating Trainer...
[+]Starting Training...
[+]Loading Data: temp/Data/All-All.csv


X: (859673, 4)
y: (859673,)


[+]Splitting Data...


2021-09-08 11:59:12.442907: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-08 11:59:12.442964: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-08 11:59:12.442994: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dhcppc10): /proc/driver/nvidia/version does not exist
2021-09-08 11:59:12.443172: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-08 11:59:12.518304: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optim

Epoch 1/2


...

15756/18806 [========================>.....] - ETA: 5s - loss: 0.6921 - accuracy: 0.4724

15787/18806 [========================>.....] - ETA: 4s - loss: 0.6921 - accuracy: 0.4724

15819/18806 [========================>.....] - ETA: 4s - loss: 0.6921 - accuracy: 0.4724

15850/18806 [========================>.....] - ETA: 4s - loss: 0.6921 - accuracy: 0.4725

KeyboardInterrupt: 